In [109]:
import time
from ont_fast5_api.fast5_interface import get_fast5_file
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import math

In [110]:
def loadReads(f5_file="../../similar_testdata/similar_squiggles.fast5", withMetadata=False):
    '''
        returns list of numpy arrays representing reads as well as dataframe containing reads metadata.
    '''
    print("reading squiggles file...")
    start = time.time()
    reads = []
    if withMetadata:
        ids = []
        length = []
        with get_fast5_file(f5_file, mode="r") as f5:
            for read in f5.get_reads():
                raw_data = read.get_raw_data()
                reads.append(raw_data)
                ids.append(read.read_id)
                length.append(len(raw_data))

        z = list(zip(ids, length))
        print(f"Took {time.time()-start} seconds.\n")
        return reads, pd.DataFrame(z, columns=["id", "length"])

    else:
        with get_fast5_file(f5_file, mode="r") as f5:
            for read in f5.get_reads():
                raw_data = read.get_raw_data()
                reads.append(raw_data)
        print(f"Took {time.time()-start} seconds.\n")
        return reads

In [111]:
def removeSpikes(sequence):
    # print("removing spikes")
    # start = time.time()
    
    spikeThresh = 2000
    returnList = [i for i in sequence if abs(i) < spikeThresh]
    # print(f"Took {time.time()-start} seconds.\n")

    return returnList

In [112]:
def zScoreNormalise(sequence):
    # print("z-score normalising")
    # start = time.time()
    
    zScoreNormalised = np.zeros(len(sequence))
    mean = np.mean(sequence)
    standardDev = np.std(sequence)
    zScoreNormalised = (sequence - mean) / standardDev
    # print(f"Took {time.time()-start} seconds.\n")

    return zScoreNormalised

In [113]:
def reduceResolution(sequence):
    #resolution reduction method based on significant changes in values
    # print("reducing resolution")
    # start = time.time()

    thresh = 0.2

    differences = []

    for i in range(len(sequence)-1):
        differences.append(abs(sequence[i+1] - sequence[i]))

        differences.insert(0, 0)		#inserting 0 at 0th index to compensate for shift

    resReducedSquiggle = []
    sig = 0
    for i in range(len(sequence)):
        if differences[i] > thresh:
            sig = sequence[i]
        resReducedSquiggle.append(sig)

    # print(f"Took {time.time()-start} seconds.\n")
    
    return resReducedSquiggle

In [114]:
def squeezeTime(sequence):
    # print("squeezing time")
    # start = time.time()
    
    timeNormalised = []
    for i in range(1, len(sequence)):
        if sequence[i] != sequence[i-1]:
            timeNormalised.append(sequence[i])

    # print(f"Took {time.time()-start} seconds.\n\n\n")
    return timeNormalised

In [115]:
def fullNormalise(sequence):
    '''
    1) spike removal
    2) z-score normalisation
    3) resolution reduction
    4) time squeezing
    '''
    s = sequence.copy()
    s = removeSpikes(s)
    s = zScoreNormalise(s)
    s = reduceResolution(s)
    s = squeezeTime(s)
    return s

In [ ]:
def normaliseDataset(squiggles, path="../../normalised/"):
    batchSize = 100
    lastBatch = False
    batchNo = 0
    totalBatches = math.floor(len(squiggles)/batchSize)
    while not lastBatch:
        print(f"Batch {batchNo} / {totalBatches}")
        if batchNo == totalBatches:
            lastBatch = True
            batch = squiggles[batchNo*batchSize:]
        else:
            batch = squiggles[batchNo*batchSize:(batchNo+1)*batchSize]

        
        for i in squiggles:
            f = h5py.File(f"{path}normalised-batch{batch}.hdf5", "a")
            dt = h5py.vlen_dtype(np.dtype("float32"))
            vectorsDset = f.create_dataset("vectors", (len(batch),), dtype=dt)

            for i in range(len(batch)):
                normalised = fullNormalise(batch[i])
                vectorsDset[i] = normalised
                vectorsList.append(normalised)            

        f.close()

        batchNo += 1


In [116]:
allSquiggles, metadata = loadReads(withMetadata=True)

reading squiggles file...
Took 5.058533430099487 seconds.



NB The slowest section by far is the "reduceResolution" function - optimisations should be first focussed on simplifying this function.

In [117]:
#normalSquiggles = [fullNormalise(i) for i in allSquiggles]

normaliseDataset(allSquiggles)

removing spikes
Took 0.2854642868041992 seconds.

z-score normalising
Took 0.026804208755493164 seconds.

reducing resolution
Took 6.522855520248413 seconds.

squeezing time
Took 0.010734081268310547 seconds.



removing spikes
Took 0.26627635955810547 seconds.

z-score normalising
Took 0.02706313133239746 seconds.

reducing resolution
Took 18.504704236984253 seconds.

squeezing time
Took 0.01617717742919922 seconds.



removing spikes
Took 0.323117733001709 seconds.

z-score normalising
Took 0.030600309371948242 seconds.

reducing resolution
Took 29.35622787475586 seconds.

squeezing time
Took 0.018991708755493164 seconds.



